In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import urllib.robotparser
from requests.exceptions import RequestException
import warnings

# Ignorer les avertissements spécifiques de BeautifulSoup
warnings.filterwarnings("ignore", category=UserWarning, module="bs4")

# Fonction pour normaliser les URLs (enlever les fragments et les barres obliques finales)
def normalize_url(url):
    parsed = urlparse(url)
    normalized = parsed._replace(fragment="").geturl()
    return normalized.rstrip('/')

# Fonction pour créer un nom de fichier valide à partir d'une URL
def url_to_filename(url):
    parsed = urlparse(url)
    path = parsed.path.replace("/", "").strip(" ")
    if not path:
        path = "index"  # Nom par défaut pour les URLs racines
    return path + ".txt"

# Fonction pour sauvegarder le contenu dans un fichier
def save_content_to_file(url, content):
    output_dir = "crawled_content"  # Répertoire pour sauvegarder les fichiers
    os.makedirs(output_dir, exist_ok=True)  # Créer le répertoire s'il n'existe pas
    filename = url_to_filename(url)
    filepath = os.path.join(output_dir, filename)
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(content)
    print(f"Contenu sauvegardé dans : {filepath}")

# Vérifier les règles du fichier robots.txt pour une URL donnée
def can_crawl(url):
    parsed_url = urlparse(url)
    robots_url = f"{parsed_url.scheme}://{parsed_url.netloc}/robots.txt"
    
    rp = urllib.robotparser.RobotFileParser()
    try:
        rp.set_url(robots_url)
        rp.read()
        return rp.can_fetch("*", url)
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier robots.txt pour {robots_url}: {e}")
        return True  # En cas d'erreur, autoriser le crawl par défaut

# Fonction pour crawler et récupérer le contenu
def crawl_and_save_content(url, visited=set(), file_limit=501, file_count=[0], keywords=None):
    if file_count[0] >= file_limit:
        return  # Arrêter si la limite de fichiers est atteinte

    normalized_url = normalize_url(url)
    if normalized_url in visited:
        return  # Ne pas revisiter les URLs déjà visitées

    visited.add(normalized_url)  # Marquer l'URL normalisée comme visitée

    # Vérifier si l'URL peut être crawled
    if not can_crawl(normalized_url):
        print(f"Le crawl est interdit pour {normalized_url} selon robots.txt.")
        return

    # Faire la requête HTTP
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    try:
        response = requests.get(normalized_url, headers=headers, timeout=10)
        
        if response.status_code == 200:
            # Vérification des CAPTCHA
            if "captcha" in response.text.lower():
                print(f"Page protégée par CAPTCHA détectée : {normalized_url}")
                return  # Ignorer cette page et continuer avec le crawl

            # Utilisation uniquement du parser HTML pour éviter les avertissements
            soup = BeautifulSoup(response.content, 'html.parser')

            # Extraire tout le texte de la page
            text_content = soup.get_text(separator="\n", strip=True)
            
            # Vérification des mots-clés
            if text_content and any(keyword in text_content for keyword in keywords):
                save_content_to_file(normalized_url, text_content)
                file_count[0] += 1  # Incrémenter le compteur de fichiers

            # Extraire et suivre tous les liens internes
            links = soup.find_all('a', href=True)
            for link in links:
                if file_count[0] >= file_limit:
                    break  # Arrêter si la limite de fichiers est atteinte
                full_url = urljoin(normalized_url, link['href'])  # Résoudre les liens relatifs
                
                # Vérifier si le lien est interne
                if full_url.startswith(url):
                    crawl_and_save_content(full_url, visited, file_limit, file_count, keywords)
        else:
            print(f"Erreur lors de l'accès à {normalized_url}: {response.status_code}")
    except RequestException as e:
        print(f"Échec de la requête pour {normalized_url}: {e}")

# Liste des URLs de départ
starting_urls = [
    "https://www.snopes.com/",
    "https://www.factcheck.org/",
    "https://factcheck.afp.com/",
    "https://beforeitsnews.com/"
]

# Liste des mots-clés à rechercher
keywords = ["fake news", "hoax", "conspiracy theory"]

# Crawler chaque site de départ
for url in starting_urls:
    print(f"Démarrage du crawling pour : {url}")
    crawl_and_save_content(url, keywords=keywords)


Démarrage du crawling pour : https://www.snopes.com/
Contenu sauvegardé dans : crawled_content\join.txt
Contenu sauvegardé dans : crawled_content\top.txt
Erreur lors de l'accès à https://www.snopes.com/category/politics/?pagenum=0: 404
Erreur lors de l'accès à https://www.snopes.com/category/Entertainment/?pagenum=0: 404
Contenu sauvegardé dans : crawled_content\fact-checktitanic-jp-morgan.txt
Contenu sauvegardé dans : crawled_content\fact-checkblack-waitress-fired-musk.txt
Contenu sauvegardé dans : crawled_content\fact-checkgaetz-ag-teens-post.txt
Contenu sauvegardé dans : crawled_content\fact-checkai-tiktok-polar-bear-cub-rescue.txt
Contenu sauvegardé dans : crawled_content\fact-checkdemocrats-osama-bin-laden-pic.txt
Contenu sauvegardé dans : crawled_content\fact-checkmusk-officially-buying-mcdonalds.txt
Contenu sauvegardé dans : crawled_content\news20241107trump-project-2025-plan.txt
Contenu sauvegardé dans : crawled_content\news20241106trump-win-2024-presidential-election.txt
Conte

In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
from requests.exceptions import RequestException

# Fonction pour normaliser les URLs (enlever les fragments et les barres obliques finales)
def normalize_url(url):
    parsed = urlparse(url)
    normalized = parsed._replace(fragment="").geturl()
    return normalized.rstrip('/')

# Fonction pour créer un nom de fichier valide à partir d'une URL
def url_to_filename(url):
    parsed = urlparse(url)
    path = parsed.path.replace("/", "").strip(" ")
    if not path:
        path = "index"  # Nom par défaut pour les URLs racines
    return path + ".txt"

# Fonction pour sauvegarder le contenu dans un fichier
def save_content_to_file(url, content):
    output_dir = "crawled"  # Répertoire pour sauvegarder les fichiers
    os.makedirs(output_dir, exist_ok=True)  # Créer le répertoire s'il n'existe pas
    filename = url_to_filename(url)
    filepath = os.path.join(output_dir, filename)
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(content)
    print(f"Contenu sauvegardé dans : {filepath}")

# Fonction pour crawler et récupérer le contenu
def crawl_and_save_content(url, visited=set(), file_limit=501, file_count=[0], keywords=None):
    if file_count[0] >= file_limit:
        return  # Arrêter si la limite de fichiers est atteinte

    normalized_url = normalize_url(url)
    if normalized_url in visited:
        return  # Ne pas revisiter les URLs déjà visitées

    visited.add(normalized_url)  # Marquer l'URL normalisée comme visitée

    # Faire la requête HTTP
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    try:
        response = requests.get(normalized_url, headers=headers, timeout=10)
        
        if response.status_code == 200:
            # Vérification des CAPTCHA
            if "captcha" in response.text.lower():
                print(f"Page protégée par CAPTCHA détectée : {normalized_url}")
                return
            
            soup = BeautifulSoup(response.content, 'html.parser')

            # Extraire tout le texte de la page
            text_content = soup.get_text(separator="\n", strip=True)
            
            # Vérification des mots-clés
            if text_content and any(keyword in text_content for keyword in keywords):
                save_content_to_file(normalized_url, text_content)
                file_count[0] += 1  # Incrémenter le compteur de fichiers

            # Extraire et suivre tous les liens internes
            links = soup.find_all('a', href=True)
            for link in links:
                if file_count[0] >= file_limit:
                    break  # Arrêter si la limite de fichiers est atteinte
                full_url = urljoin(normalized_url, link['href'])  # Résoudre les liens relatifs
                
                # Vérifier si le lien est interne
                if full_url.startswith(url):
                    crawl_and_save_content(full_url, visited, file_limit, file_count, keywords)
        else:
            print(f"Erreur lors de l'accès à {normalized_url}: {response.status_code}")
    except RequestException as e:
        print(f"Échec de la requête pour {normalized_url}: {e}")

# Liste des URLs de départ
starting_urls =["https://www.hoaxbuster.com/","https://www.lemonde.fr/les-decodeurs/","https://factuel.afp.com/","https://www.liberation.fr/checknews/"]

# Liste des mots-clés à rechercher
keywords = ["fausse information", "désinformation", "théorie du complot"]

# Crawler chaque site de départ
for url in starting_urls:
    print(f"Démarrage du crawling pour : {url}")
    crawl_and_save_content(url, keywords=keywords)
